# Data Journalism Lesson 24: The importance of text

Learn how to add important labels and annotations to your charts.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('plotly')
await micropip.install('nbformat>=4.2.0') 

In [ ]:
from IPython.display import display, HTML
import pandas as pd

# --- Simple Grading/Checking Functions ---
def display_feedback(correct, message_correct, message_incorrect):
    """Displays feedback message in an HTML div based on correctness."""
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; color: #3c763d; border: 1px solid #d6e9c6; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; color: #a94442; border: 1px solid #ebccd1; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_exercise_dataframe(inputted_df, expected_df_shape=None, expected_columns=None, check_content_sample=None):
    """Checks if the inputted_df is a DataFrame and meets basic structural expectations."""
    if not isinstance(inputted_df, pd.DataFrame):
        display_feedback(False, "", "The result is not a Pandas DataFrame.")
        return False

    if expected_df_shape is not None and inputted_df.shape[0] < expected_df_shape[0]: # Check minimum rows
        display_feedback(False, "", f"The DataFrame has {inputted_df.shape[0]} rows, but expected at least {expected_df_shape[0]}.")
        return False

    if expected_columns is not None:
        missing_cols = [col for col in expected_columns if col not in inputted_df.columns]
        if missing_cols:
            display_feedback(False, "", f"The DataFrame is missing the following columns: {missing_cols}.")
            return False

    # Simple content check (can be expanded)
    if check_content_sample is not None:
         # This is a placeholder for more complex content checks if needed.
         pass

    display_feedback(True, "DataFrame structure and basic checks passed!", "")
    return True

def check_plot_elements(fig, expected_elements):
    """Checks various elements of a Plotly figure against expected values."""
    messages = []
    all_correct = True

    # Check title
    if "title" in expected_elements:
        if fig.layout.title and fig.layout.title.text and expected_elements["title"] in fig.layout.title.text:
            messages.append("Title is correct.")
        else:
            messages.append(f"Title is incorrect or missing. Expected to contain: '{expected_elements['title']}', Got: '{fig.layout.title.text if fig.layout.title else None}'.")
            all_correct = False

    # Check subtitle (often part of title or as an annotation)
    if "subtitle" in expected_elements:
        subtitle_found = False
        if fig.layout.title and fig.layout.title.text and expected_elements["subtitle"] in fig.layout.title.text:
            subtitle_found = True
            messages.append("Subtitle is correct (found in title).")
        else:
            for anno in fig.layout.annotations:
                if anno.text == expected_elements["subtitle"]:
                    subtitle_found = True
                    messages.append("Subtitle is correct (found as annotation).")
                    break
        if not subtitle_found:
            messages.append(f"Subtitle is incorrect or not found. Expected in title or as annotation: '{expected_elements['subtitle']}'.")
            all_correct = False

    # Check axis labels
    if "xaxis_title" in expected_elements:
        if fig.layout.xaxis and fig.layout.xaxis.title and fig.layout.xaxis.title.text == expected_elements["xaxis_title"]:
            messages.append("X-axis label is correct.")
        else:
            messages.append(f"X-axis label is incorrect. Expected: '{expected_elements['xaxis_title']}', Got: '{fig.layout.xaxis.title.text if fig.layout.xaxis and fig.layout.xaxis.title else None}'.")
            all_correct = False
    if "yaxis_title" in expected_elements:
        if fig.layout.yaxis and fig.layout.yaxis.title and fig.layout.yaxis.title.text == expected_elements["yaxis_title"]:
            messages.append("Y-axis label is correct.")
        else:
            messages.append(f"Y-axis label is incorrect. Expected: '{expected_elements['yaxis_title']}', Got: '{fig.layout.yaxis.title.text if fig.layout.yaxis and fig.layout.yaxis.title else None}'.")
            all_correct = False

    # Check caption (usually an annotation)
    if "caption" in expected_elements:
        caption_found = False
        for anno in fig.layout.annotations:
            if anno.text == expected_elements["caption"]:
                caption_found = True
                break
        if caption_found:
            messages.append("Caption is present.")
        else:
            present_annos = [anno.text for anno in fig.layout.annotations]
            messages.append(f"Caption annotation not found or incorrect. Expected: '{expected_elements['caption']}'. Found annotations: {present_annos}")
            all_correct = False
            
    # Check for vline
    if "vline_x" in expected_elements:
        vline_found = any(shape.type == 'line' and shape.x0 == expected_elements["vline_x"] and shape.x1 == expected_elements["vline_x"] for shape in fig.layout.shapes if shape.type == 'line')
        if vline_found:
            messages.append(f"Vertical line at x={expected_elements['vline_x']:.2f} is present.")
        else:
            messages.append(f"Vertical line at x={expected_elements['vline_x']:.2f} is missing.")
            all_correct = False

    # Check for hline
    if "hline_y" in expected_elements:
        hline_found = any(shape.type == 'line' and shape.y0 == expected_elements["hline_y"] and shape.y1 == expected_elements["hline_y"] for shape in fig.layout.shapes if shape.type == 'line')
        if hline_found:
            messages.append(f"Horizontal line at y={expected_elements['hline_y']:.2f} is present.")
        else:
            messages.append(f"Horizontal line at y={expected_elements['hline_y']:.2f} is missing.")
            all_correct = False

    # Check for specific annotations (text labels on chart)
    if "annotations_text" in expected_elements:
        present_annotation_texts = [anno.text for anno in fig.layout.annotations if anno.text is not None]
        for expected_anno_text in expected_elements["annotations_text"]:
            if expected_anno_text in present_annotation_texts:
                messages.append(f"Annotation '{expected_anno_text}' is present.")
            else:
                messages.append(f"Annotation '{expected_anno_text}' is missing. Present annotations: {present_annotation_texts}")
                all_correct = False
                
    if all_correct:
        display_feedback(True, "All checked plot elements are correct!", "")
    else:
        feedback_msg = "Some plot elements are incorrect or missing:<ul>" + "".join([f"<li>{m}</li>" for m in messages]) + "</ul>"
        display_feedback(False, "", feedback_msg)
    return all_correct

def check_mean_calculation(calculated_value, expected_value, value_name, tolerance=1e-2):
    """Checks if a calculated mean value is close to the expected value."""
    if abs(calculated_value - expected_value) < tolerance:
        display_feedback(True, f"{value_name} calculation is correct (value: {calculated_value:.2f}).", "")
        return True
    else:
        display_feedback(False, "", f"{value_name} calculation is incorrect. Expected around {expected_value:.2f}, got {calculated_value:.2f}.")
        return False

# --- Data Loading and Initial Preparation ---
data_url = "../_static/college-cost/national.csv"

national_df = pd.read_csv(data_url)

# Filter for 'exclusive and cheap' schools
exclusivencheap_df = national_df[
    (national_df['ADM_RATE'].notna()) & (national_df['ADM_RATE'] < 0.2) & 
    (national_df['COSTT4_A'].notna()) & (national_df['COSTT4_A'] < 50000)
].copy()

# Calculate stats for glue text (rounded as in Rmd)
# Ensure ADM_RATE and COSTT4_A are numeric and handle potential NaNs before mean calculation
valid_adm_rates = national_df['ADM_RATE'].dropna()
valid_costs = national_df['COSTT4_A'].dropna()

average_admit_rate_glue = round(valid_adm_rates.mean() * 100, 0) if not valid_adm_rates.empty else 0
average_cost_glue = round(valid_costs.mean(), 0) if not valid_costs.empty else 0

# Calculate precise stats for chart lines (unrounded)
actual_average_admit_rate = valid_adm_rates.mean() if not valid_adm_rates.empty else 0
actual_average_cost = valid_costs.mean() if not valid_costs.empty else 0

# Expected values for grading Exercise 2
expected_ex2_avg_admit_rate = actual_average_admit_rate 
expected_ex2_avg_cost = actual_average_cost

# Expected values for Exercise 3 lines
ex3_vline_x_expected = actual_average_admit_rate
ex3_hline_y_expected = actual_average_cost

In [ ]:
from myst_nb import glue

glue("average_admit_rate_text", f"{average_admit_rate_glue:,.0f}", display=False)
glue("average_cost_text", f"{average_cost_glue:,.0f}", display=False)

## The Goal

In this lesson, you'll learn about the critical role of text elements in creating effective data visualizations for journalism. By the end of this tutorial, you'll understand how to craft compelling headlines, write informative subtitles, add clear axis labels, and use strategic annotations to guide readers through your charts. You'll practice applying these text elements to a scatterplot, gaining hands-on experience in transforming raw data into a story-driven visualization. These skills will enable you to create more impactful and informative graphics that effectively communicate your data stories to readers.

## Why Visualize Data?

In 2016, a group of Harvard and MIT researchers set out to answer a set of what seems like simple questions: What gets people's attention when looking at a chart? And what do they remember? 

The answer surprises people who have never made charts before. Because the answer is, at least at first, the headline on the chart.

That's right: Words. Not shapes or colors. Words. When you track people's eyes, that's where they go first, more often than not. 

"When participants were shown visualizations with titles during encoding, the titles were fixated 59% of the time," according to [Borkin et. al](http://olivalab.mit.edu/Papers/07192646.pdf). "Correspondingly, during the recall phase of the experiment, titles were the element most likely to be described if present."

So not only is the title what people are looking at first, it's the title that acts as an anchor to memory. 

In short: the headline (or title depending on who you are asking) is incredibly important. In video storytelling, there's an old saying: If you have bad audio, you have bad video. A similar saying here could be this: if you have a bad title, you have a bad graphic.

In fact, [researchers at the University of Illinois in 2018](https://www.zcliu.org/vistitles/CHI18-VisTitles.pdf) found that if you give people a chart, and you give one group a title that frames it positively and another group that frames it negatively, their interpretation of the chart will be influenced by the headline. In other words, give people a graph showing the volume of water is 50 percent. One is given a title that says something to the effect of half full, the other gets half empty, and both will walk away with very different feelings about exactly the same data.

Designers have always known that text is important, they just never had the science to back them up. No less than William Playfair in 1801 wrote "that those who do not, at the first sight, understand the manner of inspecting the Charts, will read with attention the few lines of directions facing the first Chart, after which they will find all the difficulty entirely vanish, and as much information may be obtained in five minutes as would require whole days to imprint on the memory."

Just be careful what you're imprinting.

## The Basics

We started down this road a bit in the tables tutorial, but let's repeat some important stuff from that. Text is a vital part of a good graphic. Some text is absolutely required. 

These are the pieces of a good graphic:

-   Headline
-   Chatter
-   The main body
-   Annotations
-   Labels
-   Source line
-   Credit line

The first on that list is the first for a reason. The headline is an incredibly important part of any graphic. It's often the first thing a reader will see. It's got to entice people in, tell them a little bit about what they're going to see, and help tell the story.

The second item is the chatter -- the text underneath that headline. It needs to work with the headline to further the story, drive people toward the point, maybe add some context.

The two bits of text are extremely important. Let's set up a chart and talk about how to do it wrong and how to do it better.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

The data and the chart code isn't important for you to follow along. The code is nothing special -- it's a riff off of work we've done before. Under the hood, I've loaded up all colleges and universities, and we're going to look at comparing how exclusive a school is vs. how much it costs.

Let's bring up the scatterplot with an odd group of schools -- colleges that admit fewer than 20 percent of applicants, but cost less than \$50,000 a year to attend -- as a starting place.

In [ ]:

# Create the base figure object
fig_bubble1 = go.Figure()

# Add all schools as grey points (background layer)
fig_bubble1.add_trace(go.Scatter(
    x=national_df['ADM_RATE'], 
    y=national_df['COSTT4_A'], 
    mode='markers', 
    marker=dict(color='lightgrey', size=5),
    name='All Schools (National)',
    hoverinfo='skip' # Don't show hover for these background points by default
))

# Add 'exclusive and cheap' schools as red points with labels
fig_bubble1.add_trace(go.Scatter(
    x=exclusivencheap_df['ADM_RATE'], 
    y=exclusivencheap_df['COSTT4_A'], 
    mode='markers+text', # Show both markers and text labels
    text=exclusivencheap_df['INSTNM'], # Use school name for labels
    marker=dict(color='red', size=7),
    textfont=dict(size=9, color='darkred'),
    textposition="top right",
    name='Exclusive & Relatively Inexpensive',
    customdata=exclusivencheap_df[['INSTNM', 'ADM_RATE', 'COSTT4_A']], # Data for hover
    hovertemplate = (
        "<b>%{customdata[0]}</b><br>" +
        "Admit Rate: %{x:.2%}<br>" +
        "Cost: $%{y:,.0f}" +
        "<extra></extra>" # Hides the trace name from hover
    )
))

# Basic layout updates
fig_bubble1.update_layout(
    title_text='Initial Scatterplot of College Data',
    xaxis_title='Admission Rate',
    yaxis_title='Average Annual Cost',
    height=600,
    showlegend=True
)

fig_bubble1.show()

This chart is missing major parts -- required for any chart.

To fix this, we add labels -- using `update_layout` in Plotly. Each graphic -- regardless of type -- needs a title, subtitle, caption, x and y. Your title is your headline. The subtitle is called chatter -- a sentence under the headline that explains a little about the graphic. The caption is where you'll put the source of your data and the credit line -- your name. x and y should be in each layout update, but sometimes what x and y are is obvious and you can leave them blank. For example: A bar chart with states on the y axis doesn't need to label them as States. It's obvious by the rest of the chart. In our case, nothing is obvious unless you've seen this data before. None of our readers have.

First, let's start with some headline basics:

-   Your headline should be about what the chart is about, **not what makes up the chart**. What story is the chart telling? What made it interesting to you? Don't tell me what the stats are, tell me what it says.
-   Your headline should be specific. **Generic headlines are boring and ignored**.
-   Your headline should, most often, have a verb. It's not a 100 percent requirement, but a headline without a verb means you're trying to be cute and ...
-   Your headline shouldn't be overly cute. Trying to get away with slang, a very Of The Moment cultural reference that will be forgotten soon, or some inside joke is asking for trouble.
-   Your headline should provoke a reaction.

### Exercise 1: Adding a headline, some chatter and some better labels

Let's add some text to our plot here. For your x and y labels, spell out and properly space and capitalize the initial characters of the column names. For a headline, use `Exclusive doesn't mean expensive` and for a subtitle, use this: `According to the federal Department of Education, there are seven schools who admit fewer than 20 percent of applicants but are among the most affordable.` On our x axis is the `Admit rate` and on the y axis is the `Average annual cost`. The caption should be `Source: US Department of Education | By Your Name`.

In [ ]:
fig_ex1 = go.Figure() # Start with a fresh figure for the exercise

fig_ex1.add_trace(go.Scatter(
    x=national_df['ADM_RATE'], 
    y=national_df['COSTT4_A'], 
    mode='markers', 
    marker=dict(color='lightgrey', size=5),
    name='All Schools (National)',
    hoverinfo='skip'
))

fig_ex1.add_trace(go.Scatter(
    x=exclusivencheap_df['ADM_RATE'], 
    y=exclusivencheap_df['COSTT4_A'], 
    mode='markers+text',
    text=exclusivencheap_df['INSTNM'],
    marker=dict(color='red', size=7),
    textfont=dict(size=9, color='darkred'),
    textposition="top right",
    name='Exclusive & Relatively Inexpensive',
    customdata=exclusivencheap_df[['INSTNM', 'ADM_RATE', 'COSTT4_A']],
    hovertemplate = (
        "<b>%{customdata[0]}</b><br>" +
        "Admit Rate: %{x:.2%}<br>" +
        "Cost: $%{y:,.0f}" +
        "<extra></extra>"
    )
))

# Define the text elements for the plot
title_text = "____"  # Headline
subtitle_text = "____" # Chatter
xaxis_label = "____"
yaxis_label = "____"
caption_text = "____"

# Update the layout with these elements
# Combine title and subtitle using HTML for formatting
full_title = f"{title_text}<br><sub>{subtitle_text}</sub>"

fig_ex1.update_layout(
    title_text=full_title,
    xaxis_title=xaxis_label,
    yaxis_title=yaxis_label,
    height=700, # Increased height for better label visibility
    showlegend=True,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    # Add caption as an annotation
    annotations=[
        dict(
            text=caption_text,
            showarrow=False,
            xref="paper", yref="paper",
            x=0, y=-0.1, # Positioned at the bottom left; adjust y as needed
            xanchor='left', yanchor='top',
            align='left'
        )
    ],
    margin=dict(b=80) # Add bottom margin for caption
)

# Format axes
fig_ex1.update_xaxes(tickformat='.0%') # Format x-axis as percentage
fig_ex1.update_yaxes(tickprefix='$', tickformat=',.0f') # Format y-axis as currency

fig_ex1.show()

The headline here works because it lays out a provocative premise (especially for parents). The chatter reveals what's in the chart without pointing at it -- show don't tell. Our x and y labels are clearer and let the reader know what's going on.

## Annotations

Another important part of a chart -- but not every chart -- is annotation. Sometimes, you need to label something in a chart to help the reader out. It can be a point or series of points. Or it can be regions of a chart. Let's return to our bubble chart.

Annotations also help us draw attention to things, or help the reader understand what they're looking at.

We're going to add some lines to our chart that represent the average admit rate and average cost. To get that, we first have to calculate that. It's simple -- just a `mean()` calculation on the relevant columns.

### Exercise 2: The means

If you don't remember what these columns are, look at your chart code above. They're the x (`ADM_RATE`) and y (`COSTT4_A`) in the scatterplot. Calculate their means from the `national_df` DataFrame and store them in `average_admit_rate_ex2` and `average_cost_ex2`.

In [ ]:
average_admit_rate_ex2 = national_df['____'].mean()
average_cost_ex2 = national_df['____'].mean()
# Check the calculations
check_mean_calculation(average_admit_rate_ex2, expected_ex2_avg_admit_rate, "Average Admit Rate")
check_mean_calculation(average_cost_ex2, expected_ex2_avg_cost, "Average Cost")

So what does that say? It says the average college admits about {glue:text}`average_admit_rate_text` percent of students and costs an average of ${glue:text}`average_cost_text` to attend for a year.

To add lines for the average admit rate and average cost, we're going to use Plotly's `add_hline()` and `add_vline()` methods. The averages will create lines to divide the chart into four corners. On the upper right quadrant -- lots of kids getting admitted, super expensive. On the lower left quadrant, nobody getting in, but super cheap. These methods just require an intercept value (`x` for `add_vline` and `y` for `add_hline`). We get the values for those from the averages we just made.

To label these quadrants, we're going to use `fig.add_annotation()` and we'll just put numbers in the `x` and the `y` to move them where we need them. We can get those numbers by looking at the x and y axis of our chart and guessing. After that, it's increasing or decreasing the number, depending on which direction we want to move it around.

### Exercise 3: Adding lines and labels

Note here -- I'm only labeling two sections of the chart. There is such a thing as too much. We want to tell the story here with the most efficient use of elements -- text, color, shape.

In [ ]:
fig_ex3 = go.Figure() # Start fresh for the exercise

# Add national data points (grey)
fig_ex3.add_trace(go.Scatter(
    x=national_df['ADM_RATE'], 
    y=national_df['COSTT4_A'], 
    mode='markers', 
    marker=dict(color='lightgrey', size=5),
    name='All Schools (National)',
    hoverinfo='skip'
))

# Add exclusive and cheap schools (red with labels)
fig_ex3.add_trace(go.Scatter(
    x=exclusivencheap_df['ADM_RATE'], 
    y=exclusivencheap_df['COSTT4_A'], 
    mode='markers+text', 
    text=exclusivencheap_df['INSTNM'],
    marker=dict(color='red', size=7),
    textfont=dict(size=8, color='darkred'), # Adjusted size from R's size=3
    textposition="top right",
    name='Exclusive & Relatively Inexpensive',
    customdata=exclusivencheap_df[['INSTNM', 'ADM_RATE', 'COSTT4_A']],
    hovertemplate = (
        "<b>%{customdata[0]}</b><br>" +
        "Admit Rate: %{x:.2%}<br>" +
        "Cost: $%{y:,.0f}" +
        "<extra></extra>"
    )
))

fig_ex3.add_vline(x=____, line_width=1, line_dash="dash", line_color="grey")
fig_ex3.add_hline(y=____, line_width=1, line_dash="dash", line_color="grey")

fig_ex3.add_annotation(
    x=0.87, 
    y=85000, 
    text="____", 
    showarrow=False, 
    font=dict(color="blue", size=10)
)

fig_ex3.add_annotation(
    x=0.3, 
    y=15000, 
    text="____", 
    showarrow=False, 
    font=dict(color="blue", size=10)
)

# Apply layout from Exercise 1 (title, labels, caption)
title_text_ex3 = "Exclusive doesn't mean expensive"
subtitle_text_ex3 = "According to the federal Department of Education, there are seven schools who admit fewer than 20 percent of applicants but are among the most affordable."
xaxis_label_ex3 = "Admit rate"
yaxis_label_ex3 = "Average annual cost"
caption_text_ex3 = "Source: US Department of Education | By Your Name"
full_title_ex3 = f"{title_text_ex3}<br><sub>{subtitle_text_ex3}</sub>"

fig_ex3.update_layout(
    title_text=full_title_ex3,
    xaxis_title=xaxis_label_ex3,
    yaxis_title=yaxis_label_ex3,
    height=700,
    showlegend=True,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    annotations=fig_ex3.layout.annotations + tuple([
        dict(
            text=caption_text_ex3,
            showarrow=False,
            xref="paper", yref="paper",
            x=0, y=-0.12, # Adjusted y for potentially more space needed
            xanchor='left', yanchor='top',
            align='left'
        )
    ]),
    margin=dict(b=100) # Increased bottom margin for caption and labels
)
fig_ex3.update_xaxes(tickformat='.0%')
fig_ex3.update_yaxes(tickprefix='$', tickformat=',.0f')

fig_ex3.show()

This is a good start. It's got some warts -- the grey background, a lack of any real typography, but we're getting to that. Note how I also made the dots for the cheap and exclusive schools red, the bulk of them grey and the lines blue (for annotations, lines are grey here). We're talking about color in the next tutorial.

## The Recap

Throughout this lesson, you've learned how to enhance your data visualizations with key text elements. You've practiced writing headlines that grab attention and convey the main story, crafting subtitles that provide context, and adding clear labels to guide readers through your chart. You've also explored how to use annotations to highlight key insights and divide your chart into meaningful sections. Remember, effective data journalism isn't just about the numbers — it's about using text strategically to tell a compelling story. As you continue to create visualizations, always consider how you can use headlines, labels, and annotations to make your charts more accessible and impactful for your audience.

## Terms to Know

- **Headline (Title)**: The main title of a chart that should convey the key message or story of the visualization. In Plotly, set via `layout.title.text`.
- **Chatter (Subtitle)**: A brief explanatory text below the headline that provides context or additional information about the chart. Often combined with the title using HTML (e.g., `<br><sub>Subtitle</sub>`) in Plotly.
- **Annotation**: Text or graphical elements added to a chart to highlight or explain specific data points or trends. In Plotly, added via `fig.add_annotation()` or as part of `layout.annotations`.
- **Caption**: Text below a chart that provides information about the data source and creator. Typically added as an annotation in Plotly, positioned at the bottom.
- **`fig.update_layout()`**: A core Plotly method to modify various aspects of a figure's layout, including titles, axis labels, legend, margins, and adding annotations.
- **`fig.add_annotation()`**: A Plotly method to add a text or graphical annotation to a figure at specified coordinates or relative positions.
- **`fig.add_vline()`**: A Plotly method to add a vertical line to a plot at a specified x-coordinate, often used for showing averages or thresholds.
- **`fig.add_hline()`**: A Plotly method to add a horizontal line to a plot at a specified y-coordinate, often used for showing averages or thresholds.
- **`mode='markers+text'`**: In `go.Scatter`, this mode displays both the data points (markers) and their corresponding text labels directly on the plot.
- **`textposition`**: An argument in `go.Scatter` (when `mode` includes 'text') that controls the position of text labels relative to their markers (e.g., 'top center', 'middle right').